In [1]:
import pandas as pd
import requests
import time
from pathlib import Path

In [2]:
MODEL = "model-2"
EVAL_FILE = "../../data/processed/bdm-corpus-2/stage-5/test_original.csv"
RESULT_FILE = "../../results/exp002/results_original.csv"
TEMPERATURE = 0.0

In [3]:
df = pd.read_csv(EVAL_FILE)
len(df)

500

In [4]:
def build_prompt(headline: str) -> str:
    return (
        f"{headline.strip()}\n\n"
        "Tarefa: classificar a manchete como impacto 'em alta', 'em baixa' ou 'manter' no real brasileiro. "
        "Saída autorizada: apenas um destes três rótulos. "
        "Saídas não conformes (qualquer conteúdo adicional) são inválidas."
    )

In [5]:
# === Step 3: Query Ollama ===
def query_ollama(model, prompt):
    res = requests.post("http://localhost:11434/api/chat", json={
        "model": model,
        "temperature": TEMPERATURE,
        "stream": False,
        "messages": [{"role": "user", "content": prompt}]
    })
    res.raise_for_status()
    return res.json()["message"]["content"].strip()

In [6]:
# === Step 4: Run prediction loop ===
predictions = []
for i, row in df.iterrows():
    prompt = build_prompt(row["Headline"])
    try:
        out = query_ollama(MODEL, prompt)
    except Exception as e:
        out = "Erro"
    predictions.append(out)
    time.sleep(0.25)  # reduce overload risk

In [7]:
import re
predictions = [
    re.sub(r'\s*\n+.*$', '', p.strip().lstrip("'\"").rstrip("'\"").strip())
    for p in predictions
]

In [8]:
predictions

['Manter',
 'Manter',
 'Em baixa',
 'Manter',
 'Manter',
 'Em baixa',
 'Em baixa',
 'Manter',
 'Em baixa',
 'Manter',
 'Em baixa',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Em alta',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Manter',
 'Manter',
 'Em alta',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Em baixa',
 'Em baixa',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Em baixa',
 'Em baixa',
 'Manter',
 'Manter',
 'Em baixa',
 'Em baixa',
 'Manter',
 'Em baixa',
 'Em baixa',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Em baixa',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Manter',
 'Mante

In [9]:
df["Prediction"] = predictions
df.to_csv(RESULT_FILE, index=False)
print(f"Saved predictions to {RESULT_FILE}")

Saved predictions to ../../results/exp002/results_original.csv


### Predictions

In [10]:
from sklearn.metrics import classification_report
import pandas as pd

# Reload the results file
df = pd.read_csv("../../results/exp002/results_original.csv")

df['Prediction'] = df['Prediction'].map({
    'Em baixa': 'Increase',
    'Em alta': 'Decrease',
    'Manter': 'Stable'
})

report = classification_report(
    df["Direction"], df["Prediction"],
    labels=["Increase", "Decrease", "Stable"],
    target_names=["Increase", "Decrease", "Stable"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

    Increase      0.472     0.234     0.312       214
    Decrease      0.261     0.077     0.119       155
      Stable      0.259     0.687     0.376       131

    accuracy                          0.304       500
   macro avg      0.330     0.333     0.269       500
weighted avg      0.351     0.304     0.269       500



In [11]:
from sklearn.metrics import confusion_matrix

labels = ["Increase", "Decrease", "Stable"]

cm = confusion_matrix(df["Direction"], df["Prediction"], labels=labels)
cm_df = pd.DataFrame(cm, index=[f"Actual {label}" for label in labels],
                     columns=[f"Predicted {label}" for label in labels])

print("\nConfusion Matrix:\n")
display(cm_df)


Confusion Matrix:



,Predicted Increase,Predicted Decrease,Predicted Stable
Actual Increase,50,23,141
Actual Decrease,26,12,117
Actual Stable,30,11,90
